<a href="https://colab.research.google.com/github/DanielDialektico/dialektico-machine-learning-practices/blob/main/notebooks/Machine%20Learning/Aprendizaje%20Supervisado/descenso_de_gradiente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

# **Descenso de gradiente 🌠**
---

# **Introducción**

En el recorrido [introductorio al descenso de gradiente](https://dialektico.com/introduccion-al-descenso-de-gradiente/) hemos revisado cómo es el funcionamiento interno de este artefacto para la optimización de **funciones de costo** y su uso para entrenamiento de modelos de **aprendizaje automático**.

En esta práctica veremos cómo luce la evolución de los parámetros para un nuevo modelo, operando sobre un conjunto de datos que no habíamos utilizado anteriormente.  

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/07/DGG_Colab.png" width="300" /></center>

<br>

#**Objetivo**

El objetivo de esta práctica es entrenar un modelo de aprendizaje supervisado utilizando el algoritmo de descenso de gradiente.

Debemos seguir los siguientes pasos:

* Comprender el problema y realizar un análisis exploratorio de datos.
* Aplicar preprocesamiento de datos.
* Enrenar el modelo con descenso de gradiente.
* Evaluar el modelo y generar un reporte de resultados.

Con esto comprenderemos cómo es que el descenso de gradiente es un recurso útil para el entrenamiento de modelos que hemos explorado con anterioridad.

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/08/DGG_Colab_mision.png" width="400" /></center>

<br>

# Planteamiento del problema

En esta ocasión haremos uso de nuestras habilidades como científicxs de datos y atenderemos un nuevo caso: **predicción de tarifas de taxis**.

Para esto utilizaremos un conjunto de datos con las siguientes características:
* **Distancia en kilómetros** (Trip_Distance_km): La longitud del viaje en kilómeros.
* **Momento del viaje** (Time_of_Day): Si se realizó durante la mañana, tarde o noche.
* **Día de la semana** (Day_of_Week): Día de la semana en que se efectuó el viaje.
* **Condiciones del tráfico** (Traffic_Conditions): Indicador de las caracerísticas del tráfico (ligero, medio, pesado).
* **Cantidad de pasajeros** (Passenger_Count): Número de pasajeros para el viaje.
* **Condición climática** (Weather): Datos categóricos del clima (despejado, lluvia, nieve).
* **Duración del viaje en minutos** (Trip_Duration_Minutes): Tiempo total del viaje.
* **Tarifa por kilómero en dólares** (Per_Km_Rate): La tarifa cobrada por kilómetro recorrido.
* **Tarifa cobrada por minuto en dólares** (Per_Minute_Rate): La tarifa cobrada por minuto de duración del viaje.
* **Tarifa base en dólares** (Base_Fare): La tarifa base inicial del trayecto en taxi antes de aplicar cualquier recargo por distancia o tiempo.
* **Monto de la tarifa final en dólares** (Trip_Price): El costo del viaje.

Nuestra misión es entrenar un modelo que, basado en las variables del conjunto de datos, nos permita estimar el costo de un viaje (monto de la tarifa).

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/08/DDG_D2.jpg" width="400" /></center>

¿Lo sabes? Dado que la tarifa es un número de tipo continuo, se trata de un problema de **regresión**, para lo cual podemos utilizar una regresión lineal multivariable.

<br>

#Preparación de datos y librerías

Comenzaremos por cargar las librerías que utilizaremos. Volveremos a utilizar **mlektic**, la cual nos permite aplicar descenso de gradiente de manera sencilla, y obtener un reporte automatizado de resultados (la instalación puede tardar un poco).

***Nota*:** *Las instalaciones se realizan a pesar de que algunas librerías ya están integradas de forma nativa en Colab, esto para asegurar que el Notebook no presente problemas de ejecución si se dan cambios en la sintaxis entre versiones de librerías.*

In [ ]:
# Se instalan las librerías necesarias.
!pip install mlektic==1.0.8

Se importan las librerías y se establecen configuraciones adicionales:

In [ ]:
# Se importan las librerías.
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from mlektic.linear_reg import LinearRegressionArcht
from mlektic.reporting import ReportBuilder
from mlektic.plot_utils import plot_cost
from mlektic import preprocessing, methods
from google.colab import files
import seaborn as sns
import numpy as np
import warnings


# Se filtran las advertencias.
warnings.filterwarnings('ignore')

# Se define el estilo de las gráficas.
plt.style.use('seaborn-v0_8-whitegrid')

# Se define el despliegue de flotantes en dataframes.
pd.options.display.float_format = '{:.2f}'.format

Ahora cargamos el conjunto de datos con el que trabajaremos:

In [ ]:
# Se toma el conjunto de datos de un repositorio de GitHub.
dataset = pd.read_csv('https://raw.githubusercontent.com/DanielDialektico/dialektico-machine-learning-practices/refs/heads/main/data/viajes_taxi.csv', encoding='latin1')
dataset

Ahora mostramos información general de los datos:

In [ ]:
dataset.info()

In [ ]:
# Se imprime una tabla con datos estadísticos de cada variable del conjuto de datos.
dataset.describe()

<br>

Podemos notar información de valor de manera rápida:
* Tenemos valores nulos en todas las variables, ya que deberían ser 1000 registros no nulos de cada una.
* Existen valores no numéricos.
* En algunas variables la desviación estándar es en apariencia alta.
* Los mímimos y máximos globales están bastante separados (valores más altos y más bajos del conjunto).

<br>

# **Preprocesamiento de datos**

Haremos un preprocesamiento rápido de los datos con las siguientes etapas:

* Limpieza de valores nulos.
* Transformación de datos categóricos en numéricos.
* Limpieza de valores atípicos.
* Estandarización de datos.

***Nota***: *es posible realizar diferentes análisis y etapas de preprocesamiento. Aquí lo haremos de forma sencilla para evitar extendernos en la práctica, pero eres libre de realizar las adecuaciones que consideres necesarias al conjunto de datos.*

<br>

## Limpieza de datos

Comenzaremos por mostrar el número de valores nulos, a pesar de que pudimos vislumbrarlo en las tablas antes invocadas:

In [ ]:
# Se imprimen los valores nulos de cada columna.
dataset.isnull().sum()

En promedio tenemos 50 valores nulos por variable, lo cual representa un 5% de la muestra total. Dado que queremos utilizar un proceso de limpieza simple, eliminamos los valores nulos:

In [ ]:
# Removemos los valores nulos.
dataset_wnulls = dataset.dropna()

# Mostramos los nulos.
dataset_wnulls.isnull().sum()

Ahora que hemos realizado la remoción de los datos nulos, buscaremos algún tipo de valor extraño en las variables categóricas (con valores cualitativos), mostrando sus valores únicos:

In [ ]:
# Se seleccionan columnas tipo object (valores cualitativos).
obj_cols = dataset_wnulls.select_dtypes(include="object").columns

# Se imprimen los valores únicos de cada columna.
for col in obj_cols:
    print(f"Columna: {col}")
    print(f"Valores únicos: {dataset_wnulls[col].unique()}")
    print("\n")

Hemos comprobado que los valores no contienen algún tipo de error de escritura, por lo que podemos trabajar ahora en sustituir estos por valores numéricos.

<br>

## Tansformación de datos

Para transformar los datos utilizaremos la librería [scikit-learn](https://scikit-learn.org/stable/), con la función `OrdinalEncoder`, que asignará números a cada valor cualitativo basado en un orden que le hayamos dado a las variables. En este caso, es importante el orden, ya que los valores de las variables se relacionan de esta manera, por ejemplo: la mañana es antes de la noche, y la magnitud del tráfico va de menor a mayor.

In [ ]:
# Se seleccionan las columnas categóricas a codificar.
ord_cols = ["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]

# Se define el orde de las categorías.
categories = [
    ["Morning", "Afternoon", "Evening", "Night"],  # Tiempo del día.
    ["Weekday", "Weekend"],                        # Día de la semana.
    ["Low", "Medium", "High"],                     # Condiciones del tráfico.
    ["Clear", "Rain", "Snow"],                     # Clima.
]

# Se instancia el codificador a utilizar.
enc = OrdinalEncoder(categories=categories, encoded_missing_value=-1)

# Se codifican los valores.
dataset_enc = dataset_wnulls.copy()
dataset_enc[ord_cols] = enc.fit_transform(dataset_enc[ord_cols])

# Se muestra la tabla con los valores codificados.
dataset_enc.head()

Verificamos que se haya asignado un número a cada valor categórico:

In [ ]:
# Columnas codificadas.
cols = ["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]

# Se imprimen los valores únicos.
for col in cols:
    print(f"Columna: {col}")
    print(dataset_enc[col].unique())
    print("-" * 40)

<br>

## Revisión de correlación entre variables.

Aquí añadiremos una rápido vistazo de la correlación entre las variables, esto nos permitirá ver si existen variables con una fuerte correlación, de tal forma que podamos eliminar algunas para evitar redundancia:

In [ ]:
# Se obtiene la matriz de correlación.
corr = dataset_enc.corr(method="pearson")
corr

Observamos que existe poca correlación entre las variables, por lo que mantenemos todas para ser modeladas por la regresión lineal más adelante. En este sentido de limpieza de variables, exsten métodos más avanzados como el análisis de componentes principales (ACP), pero lo dejaremos para trabajos futuros (de cualquier manera, recuerda que eres libre de aplicar el preprocesamiento que gustes a tus datos).

<br>

## Eliminación de datos atípicos

Proseguimos con la eliminación de los valores atípicos. Idealmente, habría que revisar las distribuciones de los datos para decidir qué algoritmo utilizar para su detección, pero lo haremos directamente con un método robusto llamado Median Absolute Deviation (desviación absoluta de la mediana).


Creamos nuestra función para detección de valores atípicos con numpy:

In [ ]:
def drop_outliers_mad(df, threshold=3.5):
    """
    Elimina filas con outliers usando Median Absolute Deviation (MAD).
    - threshold: valores mayores son más tolerantes (3.5 ≈ criterio habitual).
    """
    df_clean = df.copy()
    num_cols = df_clean.select_dtypes(include="number").columns
    mask = pd.Series(True, index=df_clean.index)

    for col in num_cols:
        x = df_clean[col]
        med = x.median()
        mad = (x - med).abs().median()
        if mad == 0:
            continue  # evita división por cero en columnas constantes
        robust_z = 0.6745 * (x - med).abs() / mad
        mask &= (robust_z <= threshold) | x.isna()

    return df_clean[mask]


Aplicamos la función a nuestro conjunto de datos:

In [ ]:
# Se aplica el método MAD al conjunto de datos.
no_outliers_dataset = drop_outliers_mad(dataset_enc, threshold=3.5)

# Se imprimen las dimensiones del conjunto de datos antes y después.
print("Tamaño antes de la limpieza:", dataset_enc.shape)
print("Tamaño después de la limpieza:", no_outliers_dataset.shape)

El resultado ha sido la remoción de 14 valores atípicos, lo cual representa un muy bajo porcentaje del conjunto de datos (aprox. 2.5%).

<br>

## Estandarización de datos

Un paso típico en el preprocesamiento de los datos es la estandarización, la cual consiste en una transformación que convierte los valores de cada variable a una escala comparable, haciendo que:

\begin{align}
        \text{media}=0, \text{desviación estándar=1} \tag{1.1}
    \end{align}

Cuando se utilizan métodos de optimización como descenso de gradiente, se recomienda estandarizar los datos, ya que tener las variables en una misma escala acelera la convergencia y evita pasos desbalanceados.

Procedemos a escalar los datos utilizando `StandardScaler()` de scikit-learn, donde cuidamos el no estandarizar la variable objetivo, ya que buscamos mantener las predicciones en la escala original:

In [ ]:
# Se define la variable objetivo.
target_col = "Trip_Price"

# Se separan las variables de entrada y salida.
X = no_outliers_dataset.drop(columns=[target_col])
y = no_outliers_dataset[target_col]

# Se escalan los datos.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Se reconstruye la tabla de datos.
preprocessed_dataset = pd.DataFrame(
    X_scaled,
    columns=X.columns,
    index=no_outliers_dataset.index
)

preprocessed_dataset[target_col] = y

# Se muestra la tabla.
preprocessed_dataset


Hemos concluido con las etapas de preprocesamiento, recuerda que aquí lo hemos dejado simple, pero un buen preprocesamiento implica análisis rigurosos, visualizaciones gráficas, y diversos procesos contextualizados a las características de los datos.



<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/08/DDG_K3.jpg" width="400" /></center>

<br>

# Entrenamiento del modelo

Para entrenar el modelo, utilizaremos una **regresión lineal** con mlektik, la cual utiliza [Pytorch](https://pytorch.org/) de forma subyacente para entrenar el modelo con descenso de gradiente. El descenso de gradiente se puede realizar con otras librerías, pero aquí lo haremos con mlektik por su simplicidad y transparencia en la aplicación a regresiones lineales y logísticas.

Entrenaremos al modelo eligiendo los siguientes hiperparámetros explicados en el recorrido:
* Tasa de aprendizaje = 0.05
* Iteraciones = 10

Además, utilizaremos el 80% de los datos para entrenamiento, y el 20% restante para evaluación, esto lo haremos con `preprocessing.pd_dataset` de mlektic.

In [ ]:
# Separamos las variables de entrada de la variable objetivo.
cols = preprocessed_dataset.columns.drop("Trip_Price").tolist()

# Se separa el conjunto de datos para entrenamiento y prueba.
train_set, test_set = preprocessing.pd_dataset(
    preprocessed_dataset,
    input_columns = cols,
    output_column = "Trip_Price",
    train_fraction = 0.8,
)

# Se selecciona el optimizador, en este caso, un descenso de gradiente,
# con la tasa de aprendizaje seleccionada.
optimizer = methods.optimizer_archt("sgd-standard", learning_rate=0.05)

# Se prepara la fase de entrenamiento, configurando 10 iteraciones.
linreg = LinearRegressionArcht(
    method     = "batch",
    optimizer  = optimizer,
    iterations = 10,
    metric = "r2"
)

# Se entrena el modelo.
linreg.train(train_set);

Verás que se desglosaron algunos datos sobre la ejecución, estas son las definiciones:
*   **Epoch**: época o número de iteración del entrenamiento.
*   **Loss**: valor de la función de pérdida.
*   **R2**: métrica $R^2$ para calcular el rendimiento del modelo.



Evaluamos el $R^2$ en el conjunto de pruebas:

In [ ]:
linreg.eval(test_set, 'r2')

Podemos notar que el $R^2$ indica un modelo deficiente, ya que, como hemos visto anteriormente, debería acercarse a $1$ para indicar un buen desempeño del modelo; por ello, buscaremos modificar los hiperparámetros utilizados.

<br>

# Configuración de hiperparámetros y generación de reportes

Antes de hacer modificaciones en los parámetros, quiero que aprendas cómo generar un reporte automático del entrenamiento que acabas de ejecutar. Es bastante sencillo y rápido, simplementa se utiliza la función ```ReportBuilder``` de mlektic.

Solo debes pasar el modelo que entrenaste (en este caso, `linreg`), y de manera opcional, la descripción de tu conjunto de datos y las etapas de preprocesamiento.

Lo hacemos de la siguiente manera:



In [ ]:
# Descripción opcional del conjunto de datos.
dataset_description = "Conjunto de datos de precios de viajes en Taxi"

# Descripción opcional del preprocesamiento de datos.
preprocessing_steps = [
    "Limpieza de valores nulos.",
    "Transformación de datos categóricos en numéricos.",
    "Limpieza de valores atípicos.",
    "Estandarización de datos.",
]

# Se utiliza el constructor de reportes, pasando el modelo y descripciones.
builder = ReportBuilder(
    mdl          = linreg,
    dataset_name = dataset_description,
    preprocessing = preprocessing_steps,
)

# Se genera el reporte.
builder.to_html("reporte.html")

# Se descarga.
files.download("reporte.html")
print('Reporte generado y descargado.')

El reporte debería haberse descargado automáticamente, busca en tus descargas y abre el archivo *reporte.html*, donde encontrarás algo como esto:

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/08/DDG_report.png" width="800" /></center>

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/08/DDG_D3.png" width="400" /></center>

Desglosaremos con más detalle el reporte en lo subsiguiente, por el momento, buscaremos mejorar el desempeño del modelo.

Haremos de nuevo el entrenamiento, pero modificando un hiperparámetro, el número de iteraciones:
* Tasa de aprendizaje = 0.05
* Iteraciones = 50 (nuevo valor)

In [ ]:
optimizer = methods.optimizer_archt("sgd-standard", learning_rate=0.05)

# Se entrena el modelo, seleccionando
linreg = LinearRegressionArcht(
    method     = "batch",
    optimizer  = optimizer,
    iterations = 50, # Nuevo valor
    metric     = 'r2'
)

linreg.train(train_set);

Mostramos el valor de $R^2$ en el conjunto de datos de pruebas:

In [ ]:
linreg.eval(test_set, 'r2')

Hemos notado que el desempeño ha mejorado bastante, ya que $R^2$ se acerca mucho más a $1$ de lo que se acercaba anteriormente. Esto implica que el modelo ha mejorado al añadir un mayor número de iteraciones.

Podemos observar cómo evolucionó el valor de la función de pérdida con la siguiente gráfica:

In [ ]:
%matplotlib inline

cost_history = linreg.get_cost_history()
plot_cost(cost_history, dim = (7, 5))

Puedes notar que a lo largo de las iteraciones el valor de la función de costo ha disminuido, por lo que el descenso de gradientes ha realizado bien su trabajo minimizando este valor a lo largo de las épocas.

Como último paso, genera tu reporte:

In [ ]:
# Se utiliza el constructor de reportes, pasando el modelo y descripciones.
builder = ReportBuilder(
    mdl          = linreg,
    dataset_name = dataset_description,
    preprocessing = preprocessing_steps,
)

# Se genera el reporte.
builder.to_html("reporte.html")

# Se descarga
files.download("reporte.html")
print('Reporte generado y descargado.')

<br>

# Taxonomía del reporte

Abre el reporte y revisa cada una de las secciones:

* **Detalles de ejecución**: detalles sobre el algoritmo utilizado.
* **Conjunto de Datos**: información sobre el conjunto de datos y las variables utilizadas.
* **Hiperparámetros y configuraciones**: hiperparámetros seleccionados para la ejecución del entrenamiento.
* **Resultados**: métricas de evaluación para cada subconjunto de datos.
* **Evolución de entrenamiento**: gráficas que muestran la evolución de los valores de la función de pérdida y la métrica de evaluación a lo largo de las iteraciones.


Y finalmente, dos secciones importantes para volver transparente el producto final del entrenamiento con machine learning:

* **Muestreo de parámetros durante entrenamiento**: aquí puedes ver cómo cambiaron los valores de algunos parámetros de tu modelo. Se inició con valores aleadorios y fueron modificados por el descenso de gradiente.


<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/08/DDG_report_2.png" width="800" /></center>

Y la última sección:
* **Modelo obtenido**: muestra la forma final del modelo que has entrenado y utilizas para hacer predicciones con datos nuevos.


<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/08/DDG_report_3.png" width="800" /></center>

Con estos reportes puedes observar los detalles matemáticos de tus modelos de aprendizaje automático, manteniéndote conectado con las técnicas subyacentes que generan a la inteligencia artificial, y permitiendo compartir fácilmente resúmenes de tu trabajo.

<br>

# Ejecicio (opcional)

El ejercicio adicional es bastante sencillo. Modifica los hiperparámetros de entrenamiento para ver sus efectos en el rendimiento del modelo.

El siguiente bloque entrena el modelo y genera el reporte de manera automática:

In [ ]:
optimizer = methods.optimizer_archt("sgd-standard",
                                    learning_rate=0.05) # Tasa de aprendizaje.

# Se entrena el modelo, seleccionando
linreg = LinearRegressionArcht(
    method     = "batch",
    optimizer  = optimizer,
    iterations = 50, # Número de iteraciones.
    metric     = 'r2'
)

linreg.train(train_set);

# Se utiliza el constructor de reportes, pasando el modelo y descripciones.
builder = ReportBuilder(
    mdl          = linreg,
    dataset_name = dataset_description,
    preprocessing = preprocessing_steps,
)

# Se genera el reporte.
builder.to_html("reporte.html")

# Se descarga
files.download("reporte.html")
print('\nReporte generado y descargado.')

<br>

La práctica ha concluido aquí. Has aplicado descenso de gradiente para ajustar los parámetros de una ecuación lineal, que permite estimar el precio de un viaje dado un conjunto de valores de variables.

▶ [Regresar a la lección](https://dialektico.com/descenso-de-gradiente/) 🧙

#Apéndice

## Conjunto de datos

El conjunto de datos utilizado se obtuvo de Kaggle, sus detalles pueden ser consultados en la siguiente liga: https://www.kaggle.com/datasets/denkuznetz/taxi-price-prediction/data

In [ ]:
# Dialektico Machine learning practices © 2024 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License